
# AI Script Rewriter for YouTube / Instagram Creators (Gemini-Powered)

This notebook implements a multi-agent AI system that rewrites rough creator scripts into:
- Strong hooks  
- Optimised structure  
- SEO-aware metadata  
- Tone-adjusted, ready-to-record script  
- Thumbnail caption ideas  

The agents are powered by Google Gemini via the `google-generativeai` Python SDK.



## 0. Environment Setup

Run the cell below to install the Gemini SDK (if needed) and configure your API key.

- Generate an API key from the Google AI Studio dashboard.  
- Keep it private and never hard-code it in code you share publicly.


In [1]:
!pip install -q google-generativeai

import os
from getpass import getpass
import google.generativeai as genai

# Configure your Gemini API key
if "GEMINI_API_KEY" not in os.environ or not os.environ["GEMINI_API_KEY"]:
    os.environ["GEMINI_API_KEY"] = getpass("Paste your GEMINI API key (input hidden): ")

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Choose a model: "gemini-1.5-flash" (fast, cheap) or "gemini-1.5-pro" (stronger, more expensive)
MODEL_NAME = "models/gemini-2.5-flash" # Updated to an available model
model = genai.GenerativeModel(MODEL_NAME)

print("Gemini client configured with model:", MODEL_NAME)


Paste your GEMINI API key (input hidden): ··········
Gemini client configured with model: models/gemini-2.5-flash



## 1. Core Data Models and Base Agent Class

These classes define the shared data structures and the abstract `Agent` interface used by all agents.


In [2]:

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, Any, List

@dataclass
class ScriptInput:
    topic: str
    draft_script: str
    platform: str  # "youtube" / "instagram" / "tiktok"
    tone: str      # e.g. "friendly", "hype", "professional"

@dataclass
class ScriptOutput:
    hooks: List[str]
    structured_script: str
    seo_metadata: Dict[str, Any]
    final_script: str
    thumbnail_captions: List[str]

class Agent(ABC):
    # Abstract base class for all agents in the pipeline.
    @abstractmethod
    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        '''
        Takes the current pipeline state and returns an updated dictionary.
        '''
        pass



## 2. Ingestion & Analysis Agent

One-liner: Cleans and normalises the raw draft, extracting topic, platform, tone, and key points into a structured state.


In [3]:

class IngestionAgent(Agent):
    # Basic ingestion agent. You can later extend this to use Gemini for richer key-point extraction.
    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        script_input: ScriptInput = state["input"]

        # Simple placeholder key-point extraction (could be replaced by a Gemini call)
        key_points = [line.strip("-• ").strip()
                      for line in script_input.draft_script.splitlines()
                      if line.strip()]

        state["analysis"] = {
            "topic": script_input.topic,
            "platform": script_input.platform.lower(),
            "tone": script_input.tone,
            "key_points": key_points,
            "current_script": script_input.draft_script
        }
        return state



## 3. Hook Generator Agent

One-liner: Uses Gemini to generate multiple strong, scroll-stopping hooks tailored to topic, platform and tone.


In [4]:

class HookGeneratorAgent(Agent):
    def __init__(self, model):
        self.model = model

    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        analysis = state["analysis"]
        topic = analysis["topic"]
        platform = analysis["platform"]
        tone = analysis["tone"]

        prompt = f'''You are an expert {platform} content hook writer.

Topic: {topic}
Platform: {platform}
Desired tone: {tone}

Generate 7 very strong, scroll-stopping hooks.
- Each hook max 15 words.
- No hashtags.
- Make them punchy and emotionally engaging.
Return them as a numbered list.
'''

        response = self.model.generate_content(prompt)
        text = response.text or ""

        hooks = []
        for line in text.splitlines():
            line = line.strip()
            if not line:
                continue
            # Remove leading bullets / numbers like "1." or "-"
            line = line.lstrip("-•").strip()
            if line and line[0].isdigit():
                parts = line.split(maxsplit=1)
                if len(parts) == 2:
                    line = parts[1]
            if line:
                hooks.append(line)

        state["hooks"] = hooks
        return state



## 4. Structure Optimiser Agent

One-liner: Asks Gemini to transform the rough draft into a structured script (hook, intro, sections, CTAs) in JSON form.


In [5]:

import json

class StructureOptimizerAgent(Agent):
    def __init__(self, model):
        self.model = model

    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        analysis = state["analysis"]
        draft = analysis["current_script"]
        platform = analysis["platform"]
        hooks = state.get("hooks", [])

        prompt = f'''You are a script doctor for {platform} creators.

Take this rough draft script and turn it into a clear, structured script
with the following sections:
1. Hook
2. Fast intro (why this matters)
3. Main sections (3-5 logical steps or ideas)
4. Soft CTA (engagement)
5. Strong CTA (subscribe/follow etc.)

Use one of these hooks if they fit, otherwise improve them:
{hooks}

Draft script:
"""{draft}"""

Return output in this JSON format ONLY:

{{
  "hook": "...",
  "intro": "...",
  "sections": [
    {{"title": "...", "content": "..."}}
  ],
  "cta_soft": "...",
  "cta_main": "..."
}}
'''

        response = self.model.generate_content(prompt)
        raw = response.text

        try:
            structure = json.loads(raw)
        except Exception:
            # Fallback: try to extract JSON between first { and last }
            if "{" in raw and "}" in raw:
                raw_json = raw[raw.find("{"): raw.rfind("}") + 1]
                structure = json.loads(raw_json)
            else:
                raise ValueError("Could not parse JSON structure from Gemini response:\n" + raw)

        state["structure"] = structure
        return state



## 5. SEO Keyword Injection Agent

One-liner: Uses Gemini to generate SEO keywords, titles, description and tags based on the topic and platform.


In [6]:

class SEOKeywordAgent(Agent):
    def __init__(self, model):
        self.model = model

    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        topic = state["analysis"]["topic"]
        platform = state["analysis"]["platform"]

        prompt = f'''You are an SEO specialist for {platform} content.

Topic: {topic}

1. Suggest 8-12 SEO keywords/phrases.
2. Suggest 3 video/post titles optimised for CTR.
3. Write a 2-3 paragraph description with those keywords naturally included.
4. Suggest 10-15 tags/hashtags.

Return JSON ONLY:

{{
  "keywords": ["...", "..."],
  "titles": ["...", "..."],
  "description": "...",
  "tags": ["...", "..."]
}}
'''

        response = self.model.generate_content(prompt)
        raw = response.text

        try:
            seo = json.loads(raw)
        except Exception:
            if "{" in raw and "}" in raw:
                raw_json = raw[raw.find("{"): raw.rfind("}") + 1]
                seo = json.loads(raw_json)
            else:
                raise ValueError("Could not parse SEO JSON from Gemini response:\n" + raw)

        state["seo"] = seo
        return state



## 6. Sentiment / Tonality Adjuster Agent

One-liner: Rewrites the structured script into a single flowing script while applying the creator's chosen tone.


In [7]:

class ToneAdjusterAgent(Agent):
    def __init__(self, model):
        self.model = model

    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        structure = state["structure"]
        tone = state["analysis"]["tone"]

        prompt = f'''You are a professional script editor for video creators.

Rewrite the following structured script into a single, flowing script,
keeping the structure but applying this tone:
"{tone}" (for example: hype, friendly, professional, storytelling, etc.)

Maintain:
- Same meaning
- Same order of sections

Structured script (JSON):
{json.dumps(structure)}

Return ONLY the final ready-to-read script as plain text,
with clear line breaks for each sentence or short phrase,
ready for a teleprompter.
'''

        response = self.model.generate_content(prompt)
        script_text = (response.text or "").strip()
        state["tone_adjusted"] = script_text
        return state



## 7. Final Script Assembler Agent

One-liner: Optionally formats or post-processes the tone-adjusted script (currently passes it through as final output).


In [8]:

class ScriptAssemblerAgent(Agent):
    # Currently just passes through the tone-adjusted script.
    # Extend this if you want to add [PAUSE] markers, emphasis, etc.
    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        final_script = state.get("tone_adjusted", "")
        state["final_script"] = final_script
        return state



## 8. Thumbnail Caption Generator Agent

One-liner: Uses Gemini to generate short, bold thumbnail caption options to drive clicks.


In [9]:

class ThumbnailCaptionAgent(Agent):
    def __init__(self, model):
        self.model = model

    def run(self, state: Dict[str, Any]) -> Dict[str, Any]:
        topic = state["analysis"]["topic"]
        hook = state.get("structure", {}).get("hook", "")

        prompt = f'''You are a YouTube/Instagram thumbnail copy expert.

Topic: {topic}
Main hook: {hook}

Generate 10 ultra-short thumbnail texts:
- 2-5 words each
- Very bold and emotional
- No hashtags, no emojis
- Designed for high CTR

Return them as a JSON list of strings ONLY:
["...", "..."]
'''

        response = self.model.generate_content(prompt)
        raw = response.text

        try:
            thumbs = json.loads(raw)
        except Exception:
            if "[" in raw and "]" in raw:
                raw_json = raw[raw.find("["): raw.rfind("]") + 1]
                thumbs = json.loads(raw_json)
            else:
                raise ValueError("Could not parse thumbnail caption JSON:\n" + raw)

        state["thumbnail_captions"] = thumbs
        return state



## 9. Orchestrator: Chaining All Agents

One-liner: Runs all agents in sequence to transform the raw input into hooks, SEO metadata, final script and thumbnail captions.


In [10]:

class ScriptRewriterOrchestrator:
    def __init__(self, model):
        self.agents: List[Agent] = [
            IngestionAgent(),
            HookGeneratorAgent(model),
            StructureOptimizerAgent(model),
            SEOKeywordAgent(model),
            ToneAdjusterAgent(model),
            ScriptAssemblerAgent(),
            ThumbnailCaptionAgent(model),
        ]

    def run(self, script_input: ScriptInput) -> ScriptOutput:
        state: Dict[str, Any] = {"input": script_input}
        for agent in self.agents:
            state = agent.run(state)

        return ScriptOutput(
            hooks=state.get("hooks", []),
            structured_script=json.dumps(state.get("structure", {}), indent=2),
            seo_metadata=state.get("seo", {}),
            final_script=state.get("final_script", ""),
            thumbnail_captions=state.get("thumbnail_captions", [])
        )



## 10. Demo: Run the Full Pipeline on a Sample Script

Edit the `ScriptInput` below with your own topic, draft script, platform and tone, then run the cell to see:
- Generated hooks  
- SEO metadata  
- Final ready-to-record script  
- Thumbnail caption ideas  


In [12]:
# Example usage demo
sample_input = ScriptInput(
    topic="How to add AI engineer related projects in RESUME",
    draft_script="""Want to make your resume stand out for AI Engineer roles?
    Add projects that clearly showcase your skills—like model training,
    data preprocessing, and real-world deployment. Highlight the tools you
    used such as Python, TensorFlow or PyTorch, APIs, and cloud or MLOps
    platforms. Keep each project crisp by stating the problem, the solution
    you built, and the measurable impact it created.
""",
    platform="YOUTUBE",
    tone="friendly, hype, beginner-friendly"
)

# Re-configure the model with the correct 'models/' prefix
# Listing available models to debug 'NotFound' error.
print("Listing available models:")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

# If 'gemini-1.5-flash' is not in the list above, you might need to choose a different model.
MODEL_NAME = "models/gemini-2.5-flash" # Updated to an available model
model = genai.GenerativeModel(MODEL_NAME)

orchestrator = ScriptRewriterOrchestrator(model)
output = orchestrator.run(sample_input)

print("=== GENERATED HOOKS ===")
for i, h in enumerate(output.hooks, 1):
    print(f"{i}. {h}")

print("=== SEO METADATA (titles) ===")
for t in output.seo_metadata.get("titles", []):
    print("-", t)

print("=== FINAL READY-TO-RECORD SCRIPT ===")
print(output.final_script)

print("=== THUMBNAIL CAPTION IDEAS ===")
for t in output.thumbnail_captions:
    print("-", t)


Listing available models:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2